In [86]:
from datasets import load_dataset
import pandas as pd
from ir_eval.metrics import recall, precision, hole, ndcg
from sentence_transformers import SentenceTransformer, util, CrossEncoder
from ir_eval.utils_prompt import load_prompt_text, eval_prompt, preprocess_prompt
import collections
import os
import json
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import gradio as gr
import numpy as np
import html
def highlight_tokens(query_tokens, query_weights, doc_tokens, doc_weights):
    # Convert strings to lists if inputs are space-separated strings
    if isinstance(query_tokens, str):
        query_tokens = query_tokens.split()
    if isinstance(doc_tokens, str):
        doc_tokens = doc_tokens.split()
        
    # Convert weights to numpy arrays and normalize to [0,1]
    query_weights = np.array([float(w) for w in query_weights.split()])
    query_weights = (query_weights - query_weights.min()) / (query_weights.max() - query_weights.min() + 1e-8)
    
    doc_weights = np.array([float(w) for w in doc_weights.split()])
    doc_weights = (doc_weights - doc_weights.min()) / (doc_weights.max() - doc_weights.min() + 1e-8)
    
    # Create HTML with colored tokens for query
    query_html = []
    for token, weight in zip(query_tokens, query_weights):
        color = f"rgb(255, {int(255*(1-weight))}, {int(255*(1-weight))})"
        query_html.append(f'<span style="background-color: {color}">{html.escape(token)}</span>')
    
    # Create HTML with colored tokens for document
    doc_html = []
    query_tokens_set = set(query_tokens)
    for token, weight in zip(doc_tokens, doc_weights):
        color = f"rgb(255, {int(255*(1-weight))}, {int(255*(1-weight))})"
        if token in query_tokens_set:
            doc_html.append(f'<span style="background-color: {color}"><u>{html.escape(token)}</u></span>')
        else:
            doc_html.append(f'<span style="background-color: {color}">{html.escape(token)}</span>')
    
    # Return HTML with styled containers
    return """
        <div style="margin: 10px; padding: 10px; border: 1px solid #ccc; border-radius: 5px;">
            <strong>Query:</strong><br>
            <div style="padding: 5px;">""" + " ".join(query_html) + """</div>
        </div>
        <div style="margin: 10px; padding: 10px; border: 1px solid #ccc; border-radius: 5px;">
            <strong>Document:</strong><br>
            <div style="padding: 5px;">""" + " ".join(doc_html) + """</div>
        </div>
    """

# Create Gradio interface
iface = gr.Interface(
    fn=highlight_tokens,
    inputs=[
        gr.Textbox(label="Query tokens (space-separated)"),
        gr.Textbox(label="Query weights (space-separated)"),
        gr.Textbox(label="Document tokens (space-separated)"),
        gr.Textbox(label="Document weights (space-separated)")
    ],
    outputs=gr.HTML(label="Highlighted Tokens"),
    title="Token Importance Visualizer",
    description="Enter query and document tokens with their weights to visualize importance"
)

iface.launch()

* Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
